#Task 1: Tokenization



In [1]:
# Download gutenberg dataset
# This is the dataset on which all your models will be trained
# https://drive.google.com/file/d/0B2Mzhc7popBga2RkcWZNcjlRTGM/edit

# Tokenization into sentences and words

In [51]:
import sys, re
import math
import string

'''
This function removes the punctuation marks, special symbols from the tokens
'''
def remove_punctuations(text):
    punct = ["[","]","'s","(",")","!","\"","%","$","*","&","^","=","+","`","'",","]    
    for x in punct:
        text = text.replace(x,"")
    
    for x in text:
        if x.isdigit():
            text = text.replace(x,"")
        
    text = text.replace("\n\n" , ". ")
    text = text.replace(".\n\n" , ". ")
    text = text.replace(".." , ".")
    
    p = [":--","-","\n","  ","_",",",";",":","?","/","{","}"]    
    for pp in p:
        text = text.replace(pp , " ")
    return text

In [52]:
'''
This function tokenizes the corpus into words and sentences
'''
def tokenize(text,sent,tok):
    # Given text, tokenise into sentences and words. Take into consideration various delimiters.
    text = text.lower()
    text = remove_punctuations(text)
    
    sentences = []
    s = "<SOS> "
    for word in text:
        if word.endswith('.'):
            word = word[:-1]
            s = s + word+" <EOS>"
         
            if s != "<SOS>  <EOS>":
                s = s.replace("  "," ")
                sentences.append(s.strip())
            s = "<SOS> "
        
        else:
            s = s + word
            
    for sent1 in sentences:
        if sent1 == " ":
            remove(sent1)
            
    sent.extend(sentences)        
    tokens = text.split()
    lower_tokens = []
    for x in tokens:
        lower_tokens.append(x.lower())
    punct = string.punctuation
    table = str.maketrans('','',punct)
    stripped = [w.translate(table) for w in tokens]
    tok.extend(stripped)
    return tok,sent

'''
Reading the corpus (File read operation) and calling moving further to clean the corpus and tokenize it.
'''
import glob
def get_tokens():
    sentences = []
    tokens = []
    path = '/home/ashwini/Dropbox/Sem3/NLP/P1A/Gutenberg/txt/*.txt'
    
    files = glob.glob(path)

    for name in files:
        file = open(name, 'rt')
        text = file.read()
        tokens,sentences = tokenize(text,sentences,tokens)
        
    return tokens,sentences


# Language Model

In [53]:
def unigramLangModel(words):
    global corpus_length
    
    for word in words:
        if word != start_sentence and word != end_sentence:
            freq_dict_unigram[word] = freq_dict_unigram.get(word,0) + 1
            corpus_length += 1
        
def bigramLangModel(a,sentences):
    unigramLangModel(a)
    unique_bigrams = set()
    for sentence in sentences:
        sentence = sentence.split()
        
        previous_word = None
        for word in sentence:           
            if previous_word != None:
                freq_dict_bigram[(previous_word,word)] = freq_dict_bigram.get((previous_word,word),0) + 1

            if previous_word != start_sentence and previous_word != end_sentence:
                unique_bigrams.add((previous_word,word))
            previous_word = word
    
def trigramLangModel(a,sentences):
    bigramLangModel(a,sentences)
    unique_trigrams = set()
    for sentence in sentences:
        sentence = sentence.split()
        prev1_word = None
        prev2_word = None
        for word in sentence:
            if prev1_word != None and prev2_word != None:
                freq_dict_trigram[(prev1_word,prev2_word,word)] = freq_dict_trigram.get((prev1_word,prev2_word,word),0) + 1

                if prev1_word != start_sentence and prev1_word != end_sentence:
                    unique_trigrams.add((prev1_word,prev2_word,word))
            prev1_word = prev2_word
            prev2_word = word
   

In [54]:
def eq_classes(ugrams):
    eq = {}
    for k, v in ugrams.items():
        if v not in eq:
            eq[v] = []
        eq[v].append(k)
    return eq

In [55]:
def compare_helper(l1,l2):                                           
    i=0
    j=0
    while i<len(l1) and j<len(l2):
        if(l1[i]!=l2[j]):
            return 1
        i+=1
        j+=1
   
    if(len(l1) != len(l2)):
        return 1
    return 0

def complete_ngram(ngram, n):
# Given (N-1) gram, and the value 'N', print the possibilities that complete the n-gram
# and plot them in decresing order of frequency
    temp1 = {}
    if n==2:
        temp = ngram[-(n-1):]
        for k,v in freq_dict_bigram.items():
                if compare_helper(k[:-1],temp) == 0:
                    #print(str(k[-1]) + " | " + str(v))
                    temp1[k[-1]] = v

    elif n>=3:
        temp = ngram[-2:]
        for k,v in freq_dict_trigram.items():
                if compare_helper(k[-3:-1],temp) == 0:
                    #print(":" + str(k) + ": | " + str(v))
                    temp1[k] = v
    else:
        pass
    
    print("\n\t\t---------Suggested nth word----------\n\t\t-------------------------------------\n")
    sorted_list = sorted(temp1.items(),key = itemgetter(1),reverse = True)
    for k in sorted_list:
        print(str(k[0]) + "   " + str(k[1]))

In [56]:
# only 1 of the next 3 have to be implemented

def witten_bell(n_grams):                       
  # perform Witten-Bell smoothing
    pass
def kneser_ney(n_grams):
  # perform Kneser-Ney smoothing
    pass
# only 1 of the next 2 have to be implemented


# Smoothing Techniques

In [57]:
import numpy as np

# --------------------------------- Perform Laplace smoothing ---------------------------------------------
def laplace_smoothing(num,deno,ngram):
    num += 1    
    if ngram == 2:
        deno += len(freq_dict_unigram) + 1
    elif ngram == 3:
        deno += len(freq_dict_bigram) + 1
    return num,deno

# ----------------------------- Perform Good Turing smoothing ---------------------------------------------
def good_turing(dict1):  
    
    Nr = eq_classes(dict1)
    nr_counts = {k : len(v) for k, v in Nr.items()}
    nr_probs = {k : (k*v)/float(N) for k, v in nr_counts.items()}
    sorted_nrs = sorted(nr_counts.items())
    sorted_probs = sorted(nr_probs.items())
    MAX = sorted_nrs[0][1]
    
    for r, nr in Nr.items():
        if (r+1) in Nr:
            return ((r+1) * len(Nr[r+1])) / float(N) 
        else:
            return MAX*r**-2 / float(N)
    
# --------------------------------------------------- Backoff ---------------------------------------------

def backoff(word,sent):
    score = 0.0
    
    for i in range(len(sent)):
        if sent[i]!='<SOS>' and sent[i]!='<EOS>' and sent[i-1]!='<SOS>': 
            bi_key = (sent[i - 1],sent[i])
            count_bigram = freq_dict_bigram.get(bi_key,0)
            #count_unigram = freq_dict_unigram[sent[i - 1]]
            count_unigram = freq_dict_unigram.get(sent[i - 1],0)
            if count_bigram > 0:
                score += math.log(count_bigram)
                score -= math.log(count_unigram)
            else:
                count_unigram = freq_dict_unigram[sent[i]]
                score += math.log(count_unigram + 1)
                score -= math.log(corpus_length * 2)
                score += math.log(0.4)
    print(type(score)) 
    return score



#Task 2: Unigrams and Spelling Correction

In [72]:
def Print_top10_bottom10(ngrams):
    temp =  {}
    for key,value in ngrams.items():
        temp_str = str(key)
        temp[temp_str] = value
    
    # Calculate unigram frequencies and plot them in the descending order of frequency
    temp = OrderedDict(sorted(temp.items(),key = itemgetter(1),reverse = True))       
    word = list(temp.keys())
    freq = list(temp.values())
#     plot_ngram_freqs(word,freq)
    
    # Print top 10 
    w = []
    print("\n Top 10 \n---------\n")
    w = sorted(temp.items(),key = itemgetter(1),reverse = True)
    for i in w[:10]:
        print(str(i[0]) + "  |  " + str(temp[i[0]]))
    
    print("\n Bottom 10 \n----------\n")
    for i in w[len(temp)-10:]:
        print(str(i[0]) + "  |  " + str(temp[i[0]]))
    
def plot_ngram_freqs(word,freq):

    import plotly.plotly as py
    import plotly.graph_objs as go
    
    # Create and style traces
    trace0 = go.Scatter(x = word,y = freq,name = 'Plot Ngrams_freq',line = dict(color = ('rgb(205, 12, 24)'),width = 2))   
    data = [trace0]

    # Edit the layout
    layout = dict(title = 'Plot of ngrams vs frequency',xaxis = dict(title = 'word'),yaxis = dict(title = 'frequency'),)

    fig = dict(data=data, layout=layout)
    py.iplot(fig, filename='styled-line')


# Spelling Correction

In [73]:
# --------------------------------------- Spelling checker ------------------------------------------

alphabet = set('abcdefghijklmnopqrstuvwxyz')

def probabilistic_spellcorrection(word):
#     print(a)
    char_unigram = {}
    for cu in a:
        for l in cu:
            char_unigram[l] = char_unigram.get(l,0) + freq_dict_unigram[cu]
        
    char_bigram = {}
    for cb in a:
        for index in range(0,len(cb)-1):
            char_bigram[(cb[index],cb[index+1])] = char_bigram.get((cb[index],cb[index+1]),0) + freq_dict_unigram[cb]
            
    print(char_unigram)
    print("\n\n")
    print(char_bigram)

def edit_distance(word):
    splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes    = [a + b[1:] for a, b in splits if b]
    transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
    replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
    inserts    = [a + c + b for a, b in splits for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def spell_checker(word, n_grams):
# Given a word check if the spelling is correct using your Language model
    word = word.lower()
    return (set(edit_distance(word)) & set(freq_dict_unigram.keys()))


#Task 3 : Grammaticality Test

In [74]:
'''
Build your language model - This function builds the Language Model.
It calls trigramLangModel which inside calls BigramLangModel and which again calls UnigramLangModel thus all three
models are created one by one using the tokens.
'''
def language_model(tokens,sentences):
    global freq_dict_unigram
    global freq_dict_bigram    
    trigramLangModel(tokens,sentences)

In [75]:
''' 
Here if smoothing is set True then the following smoothing techniques can be used depending on choice
1. Laplace Smoothing
2. Good Turing
3. Backoff
'''
def unigram_sentence_probability(sent):            # To calculate the score for a sentence using unigram Language model
    unigrams = []
    un = []
    unigrams,un = tokenize(sent,unigrams,un)
    sum = 0.0

    for uni in unigrams:
        pro = cal_unigram_probability(uni,True,1,sent)
        if pro!=0:
            sum += pro
    return sum

def bigram_sentence_probability(sent):              # To calculate the score for a sentence using bigram Language model
    sum = 0.0
    previous_word = None
    sent = sent.split()
    for word in sent:
        if previous_word != None:
            bigram_prob = cal_bigram_probability(previous_word,word,True,1,sent)
            if bigram_prob!=0:
                sum += bigram_prob
        previous_word = word
    return sum

def trigram_sentence_probability(sent):            # To calculate the score for a sentence using trigram Language model
    sum = 0.0
    unique_trigrams = set()
    sent = sent.split()
    prev1_word = None
    prev2_word = None
    for word in sent:
        if prev1_word != None and prev2_word != None:
            trigram_prob = cal_trigram_probability(prev1_word,prev2_word,word,True,1,sent)
            if trigram_prob!=0:
                sum += trigram_prob
        prev1_word = prev2_word
        prev2_word = word
    return sum    
  

In [76]:
''' 
Here if smoothing is set True then the following smoothing techniques can be used depending on choice
1. Laplace Smoothing
2. Good Turing
3. Backoff
This function is the utility function which actually calculates the score for the sentences according to the choice of
Language Model and Smoothing Technique.
'''

def cal_unigram_probability(word,smoothing,sm,sent):                       # Using smoothing for unigram Language model
    num = freq_dict_unigram.get(word,0)
    denum = corpus_length
    
    if smoothing:
        if sm == 1:
            num,denum = laplace_smoothing(num,denum,2)
        elif sm == 2:
            return good_turing(freq_dict_unigram)
        elif sm == 3:
            return backoff(word,sent)
    if num == 0 or denum == 0:
        return 0.0
    else:
        return float(num) / float(denum)
    
def cal_bigram_probability(prev_word,word,smoothing,sm,sent):              # Using smoothing for bigram Language model
    num = freq_dict_bigram.get((prev_word,word),0)
    denum = freq_dict_unigram.get(prev_word,0)
    
    if smoothing:
        if sm == 1:
            num,denum = laplace_smoothing(num,denum,2)
        elif sm == 2:
            return good_turing(freq_dict_unigram)
        elif sm == 3:
            return backoff(word,sent)
    if num == 0 or denum == 0:
        return 0.0
    else:
        return float(num)/float(denum)
    
def cal_trigram_probability(prev1_word,prev2_word,word,smoothing,sm,sent): # Using smoothing for trigram Language model
    num = freq_dict_trigram.get((prev1_word,prev2_word,word),0)
    denum = freq_dict_bigram.get((prev1_word,prev2_word),0)
    
    if smoothing:
        if sm == 1:                                                    
            num,denum = laplace_smoothing(num,denum,3)
        elif sm == 2:
            return good_turing(freq_dict_bigram)
        elif sm == 3:
            return backoff(word,sent)        
    if num == 0 or denum == 0:
        return 0.0
    else:
        return float(num)/float(denum)

# Main Function

In [77]:
from collections import OrderedDict
from operator import itemgetter

# Global variables and data structures are declared here which will be used throughout the program.
start_sentence = "<SOS>"                                    # Denotes the start of sentence
end_sentence = "<EOS>"                                      # Denotes the end of sentence
freq_dict_unigram = {}                                      # A dictionary maintained for unigrams and their frequency
freq_dict_bigram = {}                                       # A dictionary maintained for bigrams and their frequency
freq_dict_trigram = {}                                      # A dictionary maintained for trigrams and their frequency
corpus_length = 0                                           # It tells the number of different words in corpus
temp_dict = {}

In [78]:
'''
To tokenize the corpus into words and sentences get_tokens function is used which returns 
1. the words in list a, and 
2. the sentences in list b
'''

a,b = get_tokens()

In [79]:
'''
This function takes the tokens as inputs and build Language Models using those tokens.
The Language Models used in this program are:-
1. Unigram Lang Model
2. Bigram Lang Model
3. Trigram Lang Model
'''
language_model(a,b)

In [80]:
N = sum(freq_dict_unigram.values())

'''
To Print the Top 10 and Bottom 10 frequency words for all types of Language Models, and
Plotting frequencies vs ngrams with descending order of frequencies for all types of Language models.
'''
Print_top10_bottom10(freq_dict_unigram)
Print_top10_bottom10(freq_dict_bigram)
Print_top10_bottom10(freq_dict_trigram)



 Top 10 
---------

the  |  64949
of  |  37833
to  |  32303
and  |  28508
a  |  19388
in  |  19382
that  |  17166
i  |  16098
it  |  14746
is  |  10591

 Bottom 10 
----------

runt  |  1
absurdities  |  1
fluted  |  1
luis  |  1
honeycombed  |  1
editorialized  |  1
undisciplined  |  1
soilers  |  1
babies  |  1
voyons  |  1

 Top 10 
---------

('of', 'the')  |  10914
('in', 'the')  |  5179
('<SOS>', 'i')  |  4957
('to', 'the')  |  4371
('<SOS>', 'the')  |  3978
('<SOS>', '<EOS>')  |  2805
('to', 'be')  |  2457
('it', 'is')  |  2398
('<SOS>', 'it')  |  2244
('<SOS>', 'a')  |  2149

 Bottom 10 
----------

('can', 'use')  |  1
('rise', 'said')  |  1
('laborer', 'i')  |  1
('of', 'unscrupulous')  |  1
('henderson', 'are')  |  1
('men', 'some')  |  1
('territory', 'it')  |  1
('eyes', 'expressed')  |  1
('grant', 'drank')  |  1
('interrogatively', '<EOS>')  |  1

 Top 10 
---------

('<SOS>', 'a', '<EOS>')  |  1409
('<SOS>', 'lincoln', '<EOS>')  |  1278
('the', 'united', 'states')  |  

In [81]:
'''
It takes a word as input and then checks for the spelling whether it is correct or not according to the vocabulary 
of corpus. If the word is not from corpus then spell_checker returns a list of words which were probably misspelled.
'''
suggestions = spell_checker("mats",freq_dict_unigram)
print("\nSuggestions:  \n---------------\n" + str(suggestions) + "\n")



Suggestions:  
---------------
{'rats', 'mate', 'hats', 'mais', 'eats', 'pats', 'mass', 'mates', 'oats', 'bats', 'mars', 'maps', 'cats', 'tats', 'nats'}



In [70]:
# Sample sentences in which one is correct according to grammar but the second one is grammatically not correct
'''
GRAMMATICALITY TEST:

Here different Language models with different smoothing techniques are used which takes the sentences and returns a score
telling the sentence's correctness.
More is the score More correct is the sentence according to the corpus grammar.
Lesser is the score it means the sentence is not according to corpus grammar.
'''

text = "<SOS> It will be observed that the philosophical admonitions in the letter to his brother, Johnston, were written on the same sheet with the letter to his father <EOS>"
text1 = "<SOS> was philosophical his brother admonitions in the letter to, Johnston with the, on the some sheet were written letter to his father <EOS>"

print("\n Unigram model \n --------------\n")
print(unigram_sentence_probability(text))
print(unigram_sentence_probability(text1))

print("\n Bigram model \n --------------\n")
print(bigram_sentence_probability(text))
print(bigram_sentence_probability(text1))

print("\n Trigram model \n --------------\n")
print(trigram_sentence_probability(text))
print(trigram_sentence_probability(text1))

'''
Here if input is given as (N-1) grams then it predicts Nth gram which may come after it according to corpus.
First Parameter may be unigram,bigram,triagram 
Second Parameter tells how many grams to be considered for predicting the next nth gram.
'''
print("\n")
complete_ngram(["is","there"],3)


 Unigram model 
 --------------

0.3997798170064511
0.3025435163492585

 Bigram model 
 --------------

0.34558719409489447
0.208055697168767

 Trigram model 
 --------------

0.0004802159828236821
0.0001585872537203643



		---------Suggested nth word----------
		-------------------------------------

('is', 'there', 'any')   20
('is', 'there', 'in')   9
('is', 'there', 'a')   9
('is', 'there', 'anything')   8
('is', 'there', 'not')   5
('is', 'there', 'to')   4
('is', 'there', '<EOS>')   4
('is', 'there', 'such')   4
('is', 'there', 'no')   3
('is', 'there', 'may')   3
('is', 'there', 'then')   3
('is', 'there', 'has')   3
('is', 'there', 'is')   2
('is', 'there', 'they')   2
('is', 'there', 'can')   2
('is', 'there', 'it')   2
('is', 'there', 'so')   1
('is', 'there', 'anyone')   1
('is', 'there', 'nothing')   1
('is', 'there', 'something')   1
('is', 'there', 'one')   1
('is', 'there', 'we')   1
('is', 'there', 'just')   1
('is', 'there', 'nobody')   1
('is', 'there', 'for')   1
(